In [43]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import chardet

In [49]:
json_path = 'C:\\lee\\work\\Database\\code\\NIR_vehicle-assets\\json'
json_files = os.listdir(json_path)
png_path = 'C:\\lee\\work\\Database\\code\\NIR_vehicle-assets\\png'
save_path = 'C:\\lee\\work\\Database\\code\\NIR_vehicle-assets\\sample'

In [45]:
def vector_to_angles(vector):
            theta = np.arctan2(vector[1], vector[0])  # Yaw (좌우 회전)
            phi = np.arccos(vector[2] / np.linalg.norm(vector))  # Pitch (상하 회전)
            return np.degrees(theta), np.degrees(phi)

In [46]:
def project_to_2d(vector, length):
            x = int(vector[0] * length)
            y = int(-vector[1] * length)
            return (center[0] + x, center[1] + y)

In [47]:
def read_img(png_dir):
    img = cv2.imread(png_dir)
    if img is None:
        raise FileNotFoundError(f"Image file '{png_dir}' not found")
    return img

In [51]:
for num, json_file in enumerate(json_files):
    # print(json_file)
    json_dir = os.path.join(json_path, json_file)
    png_dir = json_dir.replace('json','png').replace('.houdini','').replace('.info','.rgb')
    
    with open(json_dir, 'r') as file:
        data = json.load(file)
        
        img = read_img(png_dir)
        
        # 스크린 공간 바운딩 박스 추출
        screen_min = data['eye_nose_mouth_bbox']['screen_space']['min']
        screen_max = data['eye_nose_mouth_bbox']['screen_space']['max']

        # 이미지 크기 가져오기
        img_height, img_width, _ = img.shape

        # 스크린 좌표를 이미지 좌표로 변환
        min_x = int(screen_min[0] * img_width)
        min_y = int(screen_min[1] * img_height)
        max_x = int(screen_max[0] * img_width)
        max_y = int(screen_max[1] * img_height)

        # 바운딩 박스를 이미지에 그리기
        cv2.rectangle(img, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
        cv2.imwrite(f'{save_path}\\bbox_{num}.png', img)
        
        img = read_img(png_dir)
    
        # 각 동공의 정보 추출
        gaze_right = data['gaze_values']['eye_right']['gaze_vector']
        gaze_left = data['gaze_values']['eye_left']['gaze_vector']

        pupil_right = data['pupil_coordinates']['pupil_right']['screen_space_pos']
        pupil_left = data['pupil_coordinates']['pupil_left']['screen_space_pos']

        # 동공 좌표를 이미지 좌표로 변환
        right_x = int(pupil_right[0] * img.shape[1])
        right_y = int(pupil_right[1] * img.shape[0])
        left_x = int(pupil_left[0] * img.shape[1])
        left_y = int(pupil_left[1] * img.shape[0])

        # 동공을 이미지에 표시
        cv2.circle(img, (right_x, right_y), 10, (0, 0, 255), -1)  # 오른쪽 눈의 동공
        cv2.circle(img, (left_x, left_y), 10, (0, 0, 255), -1)    # 왼쪽 눈의 동공
        
        cv2.imwrite(f'{save_path}\\pupil_{num}.png', img)
        
        img = read_img(png_dir)
        
        # 시선 벡터를 이미지 좌표로 변환하여 화살표 그리기
        arrow_length = 100
        arrow_color = (0, 255, 0)  # 초록색

        # 오른쪽 눈의 동공 좌표와 시선 벡터
        right_eye_screen_pos = (int(pupil_right[0] * img_width), int(pupil_right[1] * img_height))
        right_gaze_end_pos = (
            int(right_eye_screen_pos[0] + gaze_right[0] * arrow_length),
            int(right_eye_screen_pos[1] - gaze_right[1] * arrow_length)
        )
        cv2.arrowedLine(img, right_eye_screen_pos, right_gaze_end_pos, arrow_color, 2, tipLength=0.3)

        # 왼쪽 눈의 동공 좌표와 시선 벡터
        left_eye_screen_pos = (int(pupil_left[0] * img_width), int(pupil_left[1] * img_height))
        left_gaze_end_pos = (
            int(left_eye_screen_pos[0] + gaze_left[0] * arrow_length),
            int(left_eye_screen_pos[1] - gaze_left[1] * arrow_length)
        )
        cv2.arrowedLine(img, left_eye_screen_pos, left_gaze_end_pos, arrow_color, 2, tipLength=0.3)
        
        
        cv2.imwrite(f'{save_path}\\gaze_{num}.png', img)
        
        img = read_img(png_dir)

        landmarks = data['landmarks']['ibug68']


        # 랜드마크를 이미지에 표시하기
        for landmark in landmarks:
            # 랜드마크의 스크린 공간 좌표 추출하여 이미지 좌표로 변환
            landmark_x = int(landmark['screen_space_pos'][0] * img_width)
            landmark_y = int(landmark['screen_space_pos'][1] * img_height)

            # 이미지에 랜드마크 표시
            cv2.circle(img, (landmark_x, landmark_y), 3, (255, 255, 255), -1)
        
        cv2.imwrite(f'{save_path}\\ibug68_{num}.png', img)
        
        img = read_img(png_dir)
        
        kinect_data = data['landmarks']['kinect_v2']

        # 손 위치를 이미지에 표시하기
        for kinect_point in kinect_data:
            # 스크린 공간 좌표 추출하여 이미지 좌표로 변환
            screen_x = int(kinect_point['screen_space_pos'][0] * img_width)
            screen_y = int(kinect_point['screen_space_pos'][1] * img_height)

            # 이미지에 손 위치 표시
            cv2.circle(img, (screen_x, screen_y), 5, (255, 0, 255), -1)
            
        cv2.imwrite(f'{save_path}\\kinect_{num}.png', img)
        
        img = read_img(png_dir)

            
        # 주어진 변환 행렬
        transform_head2world = np.array(data['head_transform']['transform_head2world']['mat_4x4'])
        transform_head2cam = np.array(data['head_transform']['transform_head2cam']['mat_4x4'])

        # 회전 행렬 추출 (3x3 상위 행렬)
        rot_head2world = transform_head2world[:3, :3]
        rot_head2cam = transform_head2cam[:3, :3]

        # 회전 행렬에서 방향 벡터 추출
        head_direction_world = rot_head2world @ np.array([0, 0, 1])
        head_direction_cam = rot_head2cam @ np.array([0, 0, 1])

        # 방향 벡터에서 각도 계산

        angles_world = vector_to_angles(head_direction_world)
        angles_cam = vector_to_angles(head_direction_cam)
        
        # 얼굴의 중심 계산
        center_x = int((pupil_right[0] + pupil_left[0]) * img.shape[1] / 2)
        center_y = int((pupil_right[1] + pupil_left[1]) * img.shape[0] / 2)
        center = (center_x, center_y)

        # 텍스트 위치 설정
        text_position = (50, 50)

        # 텍스트 추가 (세계 좌표계 기준)
        cv2.putText(img, f'World Yaw: {angles_world[0]:.2f}', text_position, 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(img, f'World Pitch: {angles_world[1]:.2f}', (text_position[0], text_position[1] + 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)

        # 텍스트 추가 (카메라 좌표계 기준)
        cv2.putText(img, f'Camera Yaw: {angles_cam[0]:.2f}', (text_position[0], text_position[1] + 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(img, f'Camera Pitch: {angles_cam[1]:.2f}', (text_position[0], text_position[1] + 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

        length = 100  # 화살표 길이        

        # 화살표 그리기 (세계 좌표계 기준)
        end_point_world = project_to_2d(head_direction_world, length)
        cv2.arrowedLine(img, center, end_point_world, (255, 0, 0), 2)

        # 화살표 그리기 (카메라 좌표계 기준)
        end_point_cam = project_to_2d(head_direction_cam, length)
        cv2.arrowedLine(img, center, end_point_cam, (0, 0, 255), 2)
        
        cv2.imwrite(f'{save_path}\\head_{num}.png', img)
        
        img = read_img(png_dir)
        
        coco_data = data['landmarks']['coco']['whole_body']

        # 손 위치를 이미지에 표시하기
        for coco_point in coco_data:
            # 스크린 공간 좌표 추출하여 이미지 좌표로 변환
            screen_x = int(coco_point['screen_space_pos'][0] * img_width)
            screen_y = int(coco_point['screen_space_pos'][1] * img_height)

            # 이미지에 손 위치 표시
            cv2.circle(img, (screen_x, screen_y), 5, (255, 0, 255), -1)
            
        cv2.imwrite(f'{save_path}\\coco_{num}.png', img)
        
        img = read_img(png_dir)

        mediapipe_data = data['landmarks']['mediapipe']

        # 손 위치를 이미지에 표시하기
        for mediapipe_parts in mediapipe_data:
            mediapipe_part = mediapipe_data[f'{mediapipe_parts}']
            for mediapipe_point in mediapipe_part:
                # 스크린 공간 좌표 추출하여 이미지 좌표로 변환
                screen_x = int(mediapipe_point['screen_space_pos'][0] * img_width)
                screen_y = int(mediapipe_point['screen_space_pos'][1] * img_height)

                # 이미지에 위치 표시
                cv2.circle(img, (screen_x, screen_y), 5, (255, 0, 255), -1)
                
        cv2.imwrite(f'{save_path}\\mediapipe_{num}.png', img)
        
        img = read_img(png_dir)
        
                    
        mpeg_data = data['landmarks']['mpeg4']

        # 손 위치를 이미지에 표시하기
        for mpeg_point in mpeg_data:
            # 스크린 공간 좌표 추출하여 이미지 좌표로 변환
            screen_x = int(mpeg_point['screen_space_pos'][0] * img_width)
            screen_y = int(mpeg_point['screen_space_pos'][1] * img_height)

            # 이미지에 손 위치 표시
            cv2.circle(img, (screen_x, screen_y), 5, (255, 0, 255), -1)

        cv2.imwrite(f'{save_path}\\mpeg_{num}.png', img)